# FIN510 Coursework 2 - Starter Notebook

Factor Replication or Market Prediction

<figure>
<a
href="https://colab.research.google.com/github/quinfer/fin510-colab-notebooks/blob/main/labs/coursework2_scaffold.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
<figcaption>Open in Colab</figcaption>
</figure>

# FIN510 Coursework 2 - Starter Notebook

**Instructions**: This notebook provides all the code you need for a
passable submission. Simply:

1.  Choose Option A (Replication) or Option B (Prediction)
2.  Run each code cell in order
3.  Look at the outputs (tables and figures)
4.  Write your report interpreting these outputs

**What you need to change**:

-   Your data file name (line marked with `# CHANGE THIS`)
-   Your factor choice for Option A (line marked with `# CHANGE THIS`)
-   (Optional) Your predictors for Option B

**Submission**: Submit this notebook (after running all cells) with your
Word report.

## Setup: Install and Import Libraries

Run this cell first. It installs and imports everything you need.

In [ ]:
# Install required packages (only needed once)
# Uncomment the line below if running for the first time
# !pip install pandas numpy matplotlib yfinance scikit-learn statsmodels

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display settings for nice tables
pd.set_option('display.float_format', '{:.4f}'.format)

print("✓ All libraries loaded successfully")

## Step 1: Load JKP Factor Data

**Option A: Use Provided Master Dataset **

A pre-processed global master dataset is provided that includes: - All
countries (USA, UK, Germany, etc.) - Required factors: `MKT`, `SMB`,
`HML`, `MOM` - Extended factors: `RMW`, `CMA` (optional) - Additional
themes for exploration - Monthly data in wide format

The notebook will automatically download the master dataset from GitHub.
Simply run the cell below and then filter by your chosen country.

**Option B: Download Your Own Data**

If you prefer to download your own data from https://jkpfactors.com: 1.
Navigate to **Factor Returns** tab 2. Download MKT separately
(Market/Theme/Factor\* → select “Market”) 3. Download themes (select
“All 13 Themes” or specific themes) 4. Combine using the transformation
script: [convert_jkp_portal_data.py](convert_jkp_portal_data.py)

**Note**: The master dataset approach is recommended to avoid data
format issues. If you use Option B, download the conversion script and
run it locally or in Colab.

In [ ]:
# Download and load master dataset from GitHub
# This will automatically download the pre-processed dataset

import requests
import os

master_dataset_url = 'https://raw.githubusercontent.com/quinfer/fin510-colab-notebooks/main/resources/jkp_master_global_monthly.csv'
local_file = 'jkp_master_global_monthly.csv'

# Download if not already present
if not os.path.exists(local_file):
    print(f"Downloading master dataset from GitHub...")
    try:
        response = requests.get(master_dataset_url, timeout=30)
        response.raise_for_status()
        with open(local_file, 'wb') as f:
            f.write(response.content)
        print(f"✓ Downloaded: {len(response.content) / (1024*1024):.1f} MB")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print(f"   Please download manually from:")
        print(f"   https://github.com/quinfer/fin510-colab-notebooks/tree/main/resources")
        raise
else:
    print(f"✓ Using existing file: {local_file}")

# Load the data
try:
    df = pd.read_csv(local_file, parse_dates=['date'])
    print(f"✓ Data loaded: {len(df)} rows")
    print(f"\nColumns available: {df.columns.tolist()}")
    
    # Check if this is the global master dataset (has 'country' column)
    if 'country' in df.columns:
        print(f"\n✓ Global master dataset detected")
        print(f"  Countries available: {sorted(df['country'].unique())[:10]}... ({df['country'].nunique()} total)")
        if 'country_name' in df.columns:
            # Show country names for common countries
            common_codes = ['usa', 'gbr', 'deu', 'fra', 'jpn', 'chn', 'can', 'aus']
            common_countries = df[df['country'].isin(common_codes)][['country', 'country_name']].drop_duplicates()
            if len(common_countries) > 0:
                print(f"\n  Common countries:")
                for _, row in common_countries.iterrows():
                    print(f"    {row['country'].upper()}: {row['country_name']}")
        print(f"\n  Next step: Filter by country in the cell below")
    else:
        # Assume single-country wide format
        print(f"\n✓ Single-country dataset detected")
        df = df.set_index('date').sort_index()
    
    display(df.head())
except FileNotFoundError:
    print(f"❌ File '{local_file}' not found.")
    print(f"   Please ensure the download completed successfully.")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print(f"   Please check that your CSV file is correctly formatted.")

## Step 2: Select Your Country (if using global master dataset)

If you loaded the global master dataset, filter by your chosen country
here. If you’re using a single-country dataset, you can skip this step.

In [ ]:
# CHANGE THIS to your chosen country code
# Common codes: 'usa' (United States), 'gbr' (United Kingdom), 'deu' (Germany),
#                'fra' (France), 'jpn' (Japan), 'chn' (China), etc.
# 
# The dataset includes both 'country' (ISO code) and 'country_name' (full name) columns
country_code = 'usa'  # CHANGE THIS

# Filter by country (only if using global master dataset)
if 'country' in df.columns:
    df_country = df[df['country'].str.lower() == country_code.lower()].copy()
    
    if len(df_country) == 0:
        print(f"❌ No data found for country '{country_code}'")
        print(f"\nAvailable countries: {sorted(df['country'].unique())[:20]}")
    else:
        # Set date as index and sort
        df_country = df_country.set_index('date').sort_index()
        
        # Drop country columns (no longer needed after filtering)
        df_country = df_country.drop(columns=['country', 'country_name'])
        
        # Use df_country as df for rest of notebook
        df = df_country
        
        print(f"✓ Filtered to country: {country_code.upper()}")
        print(f"  Period: {df.index[0]} to {df.index[-1]}")
        print(f"  Number of months: {len(df)}")
        print(f"  Columns: {df.columns.tolist()}")
        
        # Validate required columns
        required_cols = ['MKT', 'SMB', 'HML', 'MOM']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            print(f"\n⚠️  Warning: Missing columns: {missing_cols}")
        else:
            print(f"\n✓ All required columns present: {required_cols}")
            display(df.head())
else:
    print("✓ Single-country dataset - no filtering needed")
    print(f"  Columns: {df.columns.tolist()}")

------------------------------------------------------------------------

# CHOOSE YOUR OPTION

**Run EITHER Option A OR Option B below** (not both)

------------------------------------------------------------------------

# OPTION A: Factor Replication

This option analyses a single factor (like Value or Momentum) and tests
whether it generates excess returns.

## A1: Choose Your Factor

In [ ]:
# CHANGE THIS to your chosen factor
# Options: 'SMB' (size), 'HML' (value), 'MOM' (momentum), 'RMW' (profitability), 'CMA' (investment)
factor_name = 'HML'  # CHANGE THIS

# Extract the factor and market returns
factor_returns = df[factor_name].dropna()
market_returns = df['MKT'].dropna()

# Align dates
common_dates = factor_returns.index.intersection(market_returns.index)
factor_returns = factor_returns[common_dates]
market_returns = market_returns[common_dates]

print(f"✓ Analyzing {factor_name} factor")
print(f"  Period: {factor_returns.index[0]} to {factor_returns.index[-1]}")
print(f"  Number of observations: {len(factor_returns)}")

## A2: Summary Statistics

This produces Table 1 for your report.

In [ ]:
# Calculate summary statistics
def calculate_stats(returns):
    stats = {
        'Mean (%)': returns.mean() * 100,
        'Std Dev (%)': returns.std() * 100,
        'Sharpe Ratio': (returns.mean() / returns.std()) * np.sqrt(12),
        'Min (%)': returns.min() * 100,
        'Max (%)': returns.max() * 100,
        'Skewness': returns.skew(),
        'Kurtosis': returns.kurtosis()
    }
    return pd.Series(stats)

# Create summary table
summary_table = pd.DataFrame({
    factor_name: calculate_stats(factor_returns),
    'MKT': calculate_stats(market_returns)
})

print("=== TABLE 1: Summary Statistics ===")
display(summary_table)

# Save for your report
summary_table.to_csv('table1_summary_stats.csv')
print("✓ Saved as 'table1_summary_stats.csv'")

## A3: Cumulative Returns Plot

This produces Figure 1 for your report.

In [ ]:
# Calculate cumulative returns
cum_factor = (1 + factor_returns).cumprod()
cum_market = (1 + market_returns).cumprod()

# Create plot
plt.figure(figsize=(12, 6))
plt.plot(cum_factor.index, cum_factor.values, label=factor_name, linewidth=2)
plt.plot(cum_market.index, cum_market.values, label='Market (MKT)', linewidth=2, alpha=0.7)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return (£1 invested)', fontsize=12)
plt.title(f'Cumulative Returns: {factor_name} vs Market', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figure1_cumulative_returns.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved as 'figure1_cumulative_returns.png'")

## A4: Alpha Regression (CAPM)

This tests whether your factor has significant excess returns (alpha)
after controlling for market risk.

This produces Table 2 for your report.

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.stats.sandwich_covariance import cov_hac

# Prepare data for regression
Y = factor_returns.values
X = add_constant(market_returns.values)

# Run OLS regression
model = OLS(Y, X).fit()

# Get HAC (robust) standard errors
cov_hac_matrix = cov_hac(model)
se_hac = np.sqrt(np.diag(cov_hac_matrix))
t_stats_hac = model.params / se_hac

# Create results table
alpha_table = pd.DataFrame({
    'Coefficient': model.params,
    'Std Error (HAC)': se_hac,
    't-statistic': t_stats_hac,
    'p-value': [2 * (1 - np.abs(t)) for t in t_stats_hac]
}, index=['Alpha', 'Market Beta'])

# Add R-squared
alpha_table.loc['R-squared', 'Coefficient'] = model.rsquared

print("=== TABLE 2: CAPM Regression Results ===")
print(f"Regression: {factor_name} = Alpha + Beta * MKT + error")
display(alpha_table)

# Interpretation
alpha = model.params[0]
alpha_tstat = t_stats_hac[0]
print(f"📊 INTERPRETATION:")
print(f"   Alpha = {alpha*100:.3f}% per month ({alpha*12*100:.2f}% annualized)")
if abs(alpha_tstat) > 1.96:
    print(f"✓ Alpha is statistically significant (|t| = {abs(alpha_tstat):.2f} > 1.96)")
else:
    print(f"✗ Alpha is NOT statistically significant (|t| = {abs(alpha_tstat):.2f} < 1.96)")

# Save
alpha_table.to_csv('table2_alpha_regression.csv')
print("✓ Saved as 'table2_alpha_regression.csv'")

## A5: Robustness Check - Sample Split

This splits your data into two periods to check if results are stable.

This produces Table 3 and Figure 2 for your report.

In [ ]:
# Split sample at midpoint
midpoint = len(factor_returns) // 2
dates = factor_returns.index

period1_factor = factor_returns.iloc[:midpoint]
period2_factor = factor_returns.iloc[midpoint:]
period1_market = market_returns.iloc[:midpoint]
period2_market = market_returns.iloc[midpoint:]

# Calculate statistics for each period
def period_stats(factor_ret, market_ret):
    # Regression
    X = add_constant(market_ret.values)
    model = OLS(factor_ret.values, X).fit()
    cov = cov_hac(model)
    se = np.sqrt(np.diag(cov))
    t_stat = model.params[0] / se[0]
    
    return {
        'Mean Return (%)': factor_ret.mean() * 100,
        'Std Dev (%)': factor_ret.std() * 100,
        'Sharpe Ratio': (factor_ret.mean() / factor_ret.std()) * np.sqrt(12),
        'Alpha (%)': model.params[0] * 100,
        'Alpha t-stat': t_stat,
        'Market Beta': model.params[1]
    }

robustness_table = pd.DataFrame({
    f'Period 1 ({dates[0].year}-{dates[midpoint-1].year})': period_stats(period1_factor, period1_market),
    f'Period 2 ({dates[midpoint].year}-{dates[-1].year})': period_stats(period2_factor, period2_market)
})

print("=== TABLE 3: Robustness Check (Sample Split) ===")
display(robustness_table)

# Save
robustness_table.to_csv('table3_robustness.csv')

# Plot rolling Sharpe ratio
window = 36  # 3-year window
rolling_sharpe = factor_returns.rolling(window).apply(
    lambda x: (x.mean() / x.std()) * np.sqrt(12)
)

plt.figure(figsize=(12, 6))
plt.plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Rolling Sharpe Ratio (3-year)', fontsize=12)
plt.title(f'{factor_name} Factor: Rolling Sharpe Ratio', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figure2_rolling_sharpe.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved as 'table3_robustness.csv' and 'figure2_rolling_sharpe.png'")

## A6: Summary - What You Need for Your Report

You now have:

**Tables**:

1.  `table1_summary_stats.csv` - Descriptive statistics
2.  `table2_alpha_regression.csv` - CAPM alpha test
3.  `table3_robustness.csv` - Sample split results

**Figures**:

1.  `figure1_cumulative_returns.png` - Performance over time
2.  `figure2_rolling_sharpe.png` - Stability check

**For your report**, interpret these results:

-   Does your factor generate positive average returns?
-   Is the alpha statistically significant?
-   Are results stable across sub-periods?
-   What are the portfolio/investment implications?
-   What are the limitations of this analysis?

------------------------------------------------------------------------

# OPTION B: Market Prediction

This option predicts next month’s market return using factors as
predictors.

*Note: Only complete Option B if you did NOT complete Option A*

## B1: Prepare Predictors

In [ ]:
# Choose predictors (you can change these)
predictor_names = ['SMB', 'HML', 'MOM']  # CHANGE THIS if you want different predictors

# Create lagged predictors and next-month target
data = df[['MKT'] + predictor_names].dropna()

# Target: next month's market return
target = data['MKT'].shift(-1)

# Predictors: current month values
predictors = data[predictor_names]

# Also add lagged market return as predictor
predictors['MKT_lag'] = data['MKT']

# Align and remove last row (no future target)
predictors = predictors[:-1]
target = target[:-1].dropna()

# Align dates
common_dates = predictors.index.intersection(target.index)
X = predictors.loc[common_dates]
y = target.loc[common_dates]

print(f"✓ Prediction setup complete")
print(f"  Predictors: {X.columns.tolist()}")
print(f"  Sample size: {len(y)} months")
print(f"  Period: {y.index[0]} to {y.index[-1]}")

## B2: Walk-Forward Prediction

This compares OLS vs Ridge regression in a realistic walk-forward test.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score

# Settings
min_train = 120  # Minimum 10 years of training data
train_window = 120  # Use 10-year rolling window

# Storage for predictions
predictions_ols = []
predictions_ridge = []
actuals = []
dates_oos = []

# Walk-forward loop
for i in range(min_train, len(y)):
    # Training data (rolling window)
    train_start = max(0, i - train_window)
    X_train = X.iloc[train_start:i]
    y_train = y.iloc[train_start:i]
    
    # Test data (next month)
    X_test = X.iloc[i:i+1]
    y_test = y.iloc[i]
    
    # OLS model
    ols = LinearRegression().fit(X_train, y_train)
    pred_ols = ols.predict(X_test)[0]
    
    # Ridge model
    ridge = Ridge(alpha=1.0).fit(X_train, y_train)
    pred_ridge = ridge.predict(X_test)[0]
    
    # Store
    predictions_ols.append(pred_ols)
    predictions_ridge.append(pred_ridge)
    actuals.append(y_test)
    dates_oos.append(y.index[i])

# Convert to series
pred_ols_series = pd.Series(predictions_ols, index=dates_oos)
pred_ridge_series = pd.Series(predictions_ridge, index=dates_oos)
actual_series = pd.Series(actuals, index=dates_oos)

print(f"✓ Walk-forward prediction complete")
print(f"  Out-of-sample period: {dates_oos[0]} to {dates_oos[-1]}")
print(f"  Number of predictions: {len(actuals)}")

## B3: Evaluation Metrics

This produces Table 1 for your report.

In [ ]:
# Calculate metrics
def eval_metrics(actual, predicted):
    # R-squared OOS
    ss_res = ((actual - predicted) ** 2).sum()
    ss_tot = ((actual - actual.mean()) ** 2).sum()
    r2_oos = 1 - (ss_res / ss_tot)
    
    # Directional accuracy
    direction_correct = (np.sign(actual) == np.sign(predicted)).sum()
    direction_acc = direction_correct / len(actual) * 100
    
    # RMSE
    rmse = np.sqrt(((actual - predicted) ** 2).mean())
    
    return {
        'R² OOS (%)': r2_oos * 100,
        'Direction Accuracy (%)': direction_acc,
        'RMSE': rmse,
        'Mean Absolute Error': (actual - predicted).abs().mean()
    }

# Benchmark: historical mean
benchmark_pred = pd.Series([actual_series.mean()] * len(actual_series), index=actual_series.index)

# Compile results
results_table = pd.DataFrame({
    'OLS': eval_metrics(actual_series, pred_ols_series),
    'Ridge': eval_metrics(actual_series, pred_ridge_series),
    'Historical Mean': eval_metrics(actual_series, benchmark_pred)
})

print("=== TABLE 1: Prediction Performance ===")
display(results_table)

# Save
results_table.to_csv('table1_prediction_results.csv')
print("✓ Saved as 'table1_prediction_results.csv'")

## B4: Rolling R² Plot

This produces Figure 1 for your report.

In [ ]:
# Calculate rolling R² (12-month window)
def rolling_r2(actual, predicted, window=12):
    r2_values = []
    dates = []
    
    for i in range(window, len(actual)):
        y_true = actual[i-window:i]
        y_pred = predicted[i-window:i]
        
        ss_res = ((y_true - y_pred) ** 2).sum()
        ss_tot = ((y_true - y_true.mean()) ** 2).sum()
        r2 = 1 - (ss_res / ss_tot)
        
        r2_values.append(r2)
        dates.append(actual.index[i])
    
    return pd.Series(r2_values, index=dates)

rolling_r2_ols = rolling_r2(actual_series, pred_ols_series)
rolling_r2_ridge = rolling_r2(actual_series, pred_ridge_series)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(rolling_r2_ols.index, rolling_r2_ols.values * 100, label='OLS', linewidth=2)
plt.plot(rolling_r2_ridge.index, rolling_r2_ridge.values * 100, label='Ridge', linewidth=2, alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5, label='Zero')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Rolling R² OOS (%)', fontsize=12)
plt.title('Market Prediction: Rolling 12-Month R² Out-of-Sample', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figure1_rolling_r2.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved as 'figure1_rolling_r2.png'")

## B5: Summary - What You Need for Your Report

You now have:

**Tables**:

1.  `table1_prediction_results.csv` - Performance metrics

**Figures**:

1.  `figure1_rolling_r2.png` - Stability over time

**For your report**, interpret these results:

-   Which model performs better (OLS or Ridge)?
-   Is the predictive power positive (R² \> 0)?
-   Is directional accuracy better than 50% (random guessing)?
-   How stable are the predictions over time?
-   What are the investment implications?
-   What are the limitations?

------------------------------------------------------------------------

# Final Checklist

Before submitting:

**Notebook**:

-   [ ] All cells have been run successfully
-   [ ] All outputs (tables/figures) are visible
-   [ ] Data file name is correct
-   [ ] Factor choice (Option A) or predictors (Option B) are
    appropriate

**Report (separate Word document)**:

-   [ ] Introduction: Research question and motivation
-   [ ] Data: Source, period, any limitations
-   [ ] Methodology: Brief description of approach
-   [ ] Results: Include tables/figures from notebook with
    interpretation
-   [ ] Discussion: Limitations and investment implications
-   [ ] References: Cite Jensen, Kelly & Pedersen (2024) and other
    sources

**Files to submit**:

1.  This notebook (`.ipynb` file)
2.  Your report (Word document)
3.  (Optional) The data CSV file if \< 5MB

Good luck! 🎓